# 3_Convert_Scientific_Notation_to_Meters.ipynb

Este notebook processa o arquivo JSON gerado (`Dataset.json`) e:
1. Converte `diferencalatitude` e `diferencalongitude` de graus para metros.
2. Elimina a notação científica ao trabalhar com valores maiores.
3. Salva o JSON resultante em um novo arquivo chamado `Dataset_Converted.json`.

In [17]:
import json
from pathlib import Path

# Configuração dos caminhos

In [18]:
current_dir = Path().resolve()  # Diretório onde o notebook está sendo executado
base_dir = current_dir.parent / "TABELAS"  # Diretório onde está localizado o JSON original
input_json_path = base_dir / "JSONS" / "Dataset.json"  # Caminho do JSON original
output_json_path = base_dir / "JSONS" / "Dataset_Converted.json"  # Caminho do JSON convertido


# Constantes de conversão

In [19]:
METROS_POR_GRAU_LATITUDE = 111320  # 1 grau de latitude em metros
METROS_POR_GRAU_LONGITUDE = 111320  # Aproximação para simplificar (varia com latitude)


# Função para converter diferenças de graus para metros

    Converte um valor de graus para metros.
    
    :param value: Valor em graus.
    :param is_latitude: True se for latitude; False para longitude.
    :return: Valor convertido em metros.


In [20]:

def convert_to_meters(value, is_latitude=True):
    
    fator = METROS_POR_GRAU_LATITUDE if is_latitude else METROS_POR_GRAU_LONGITUDE
    return round(value * fator, 2)  # Arredonda para 2 casas decimais


# Função para converter campos específicos
Converte os campos do JSON:
    - `diferencalatitude` e `diferencalongitude`: De graus para metros.
    - Renomeia os campos para `diferencalatitudeMetros` e `diferencalongitudeMetros`.
    - `latitudereal` e `longitudereal`: Arredonda para 6 casas decimais.
    
    :param data: Lista de registros do JSON.
    :return: Lista de registros convertidos.

In [21]:
def convert_fields(data):
    for record in data:
        # Converte diferencalatitude para metros e renomeia o campo
        if "diferencalatitude" in record and record["diferencalatitude"] is not None:
            record["diferencalatitudeMetros"] = convert_to_meters(float(record["diferencalatitude"]), is_latitude=True)
            del record["diferencalatitude"]  # Remove o campo antigo
        
        # Converte diferencalongitude para metros e renomeia o campo
        if "diferencalongitude" in record and record["diferencalongitude"] is not None:
            record["diferencalongitudeMetros"] = convert_to_meters(float(record["diferencalongitude"]), is_latitude=False)
            del record["diferencalongitude"]  # Remove o campo antigo
        
        # Arredonda latitudereal e longitudereal para 6 casas decimais
        if "latitudereal" in record and record["latitudereal"] is not None:
            record["latitudereal"] = round(float(record["latitudereal"]), 6)
        
        if "longitudereal" in record and record["longitudereal"] is not None:
            record["longitudereal"] = round(float(record["longitudereal"]), 6)
        
        # Reorganiza os campos, garantindo que `diferencalatitudeMetros` e `diferencalongitudeMetros` fiquem no topo
        reordered_record = {
            "diferencalatitudeMetros": record.pop("diferencalatitudeMetros", None),
            "diferencalongitudeMetros": record.pop("diferencalongitudeMetros", None),
            **record  # Insere os outros campos na ordem original
        }
        record.clear()
        record.update(reordered_record)  # Atualiza o dicionário com a nova ordem
    
    return data

# Leitura do arquivo JSON original

In [22]:
with open(input_json_path, "r", encoding="utf-8") as input_file:
    data = json.load(input_file)

# Converte os campos

In [23]:
converted_data = convert_fields(data)

# Salva o JSON convertido com o encoder customizado

In [24]:
with open(output_json_path, "w", encoding="utf-8") as output_file:
    json.dump(converted_data, output_file, ensure_ascii=False, indent=4)